# Hybrid Retriever with BM25

이 노트북은 BM25와 벡터 검색을 결합한 하이브리드 검색 시스템을 구현하는 방법을 다룹니다.

## 1. 필요한 패키지 설치

BM25 키워드 검색을 위한 `rank_bm25` 패키지를 설치합니다.


In [ ]:
%pip install rank_bm25

## 2. BM25Retriever 기본 사용법

LangChain의 BM25Retriever를 사용하여 키워드 기반 검색을 수행합니다.
- 샘플 문서를 생성하고 BM25 인덱스를 구축
- 한국어 쿼리로 검색하여 관련 문서를 반환


In [1]:
from langchain_core.documents import Document
from langchain_community.retrievers import BM25Retriever

# 샘플 문서 생성
documents = [
    Document(page_content="Langchain은 LLM 애플리케이션 개발을 위한 프레임워크입니다."),
    Document(page_content="Qdrant는 고성능 벡터 데이터베이스입니다."),
    Document(page_content="BM25는 정보 검색에서 사용되는 랭킹 함수입니다."),
    Document(page_content="RAG는 검색 증강 생성 방법론입니다."),
    Document(page_content="키워드 검색과 벡터 검색을 결합하면 더 좋은 결과를 얻을 수 있습니다."),
]

# BM25Retriever 생성
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 3  # 반환할 문서 수 설정

# 검색 실행
query = "벡터 데이터베이스"
results = bm25_retriever.invoke(query)

print(f"🔍 검색 쿼리: '{query}'")
print(f"📊 검색 결과 ({len(results)}개):")
for i, doc in enumerate(results, 1):
    print(f"{i}. {doc.page_content}")

🔍 검색 쿼리: '벡터 데이터베이스'
📊 검색 결과 (3개):
1. Qdrant는 고성능 벡터 데이터베이스입니다.
2. 키워드 검색과 벡터 검색을 결합하면 더 좋은 결과를 얻을 수 있습니다.
3. RAG는 검색 증강 생성 방법론입니다.


## 3. 커스텀 전처리 함수 적용

BM25의 성능을 향상시키기 위해 커스텀 전처리 함수를 적용합니다.
- 특수문자 제거 및 소문자 변환
- 단어 토큰화를 통한 검색 품질 개선
- 동일한 쿼리로 전처리 효과 비교


In [2]:
def custom_preprocessor(text: str):
    """
    커스텀 전처리 함수
    - 소문자 변환
    - 단어 토큰화
    - 특수문자 제거
    """
    import re
    # 특수문자 제거 및 소문자 변환
    text = re.sub(r'[^\w\s]', '', text.lower())
    # 공백으로 분할
    tokens = text.split()
    return tokens

# 커스텀 전처리가 적용된 BM25Retriever
custom_bm25_retriever = BM25Retriever.from_documents(
    documents,
    preprocess_func=custom_preprocessor,
    k=3
)

# 검색 실행
results = custom_bm25_retriever.invoke("데이터베이스 벡터")
print("🛠️ 커스텀 전처리 적용 결과:")
for i, doc in enumerate(results, 1):
    print(f"{i}. {doc.page_content}")

🛠️ 커스텀 전처리 적용 결과:
1. Qdrant는 고성능 벡터 데이터베이스입니다.
2. 키워드 검색과 벡터 검색을 결합하면 더 좋은 결과를 얻을 수 있습니다.
3. RAG는 검색 증강 생성 방법론입니다.


## 4. Qdrant 벡터 데이터베이스 연결

하이브리드 검색을 위해 Qdrant 벡터 데이터베이스에 연결합니다.
- 로컬 Qdrant 서버 연결 시도
- 연결 실패 시 메모리 모드로 대체
- 기존 컬렉션 정보 확인


In [3]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.models import Distance, VectorParams


# Qdrant 클라이언트 초기화 (로컬 서버 가정)
try:
    client = QdrantClient("localhost", port=6333)
    print("✅ Qdrant 서버에 연결되었습니다.")
    
    # 서버 정보 확인
    print(f"📊 Qdrant 버전: {client.get_collections()}")
except Exception as e:
    print(f"❌ Qdrant 연결 실패: {e}")
    print("💡 Docker로 Qdrant 서버를 시작하세요: docker run -p 6333:6333 qdrant/qdrant")
    
    # 메모리 모드로 대체
    client = QdrantClient(":memory:")
    print("🔄 메모리 모드로 Qdrant를 실행합니다.")

✅ Qdrant 서버에 연결되었습니다.
📊 Qdrant 버전: collections=[CollectionDescription(name='selfquery'), CollectionDescription(name='scalar_quantization'), CollectionDescription(name='sparse_collection'), CollectionDescription(name='product_quantization'), CollectionDescription(name='agentic_retrieval_medical'), CollectionDescription(name='arxiv_papers'), CollectionDescription(name='agentic_retrieval_law'), CollectionDescription(name='agentic_retrieval_commerce'), CollectionDescription(name='physical_ai_report_mm'), CollectionDescription(name='pdf_rerank'), CollectionDescription(name='physical_ai_report_mm_retriever'), CollectionDescription(name='physical_ai_report_mm_01'), CollectionDescription(name='arxiv_physics_muon_paper'), CollectionDescription(name='hybrid_search_collection'), CollectionDescription(name='ppt_rag'), CollectionDescription(name='pdf_hybrid'), CollectionDescription(name='product_quantization_2'), CollectionDescription(name='agentic_retrieval_public'), CollectionDescription(name='fi

## 5. PDF 문서 로딩

실제 문서 데이터를 로딩하여 하이브리드 검색 시스템에 활용합니다.
- 지정된 AI 관련 PDF 파일들을 로딩
- PyPDFLoader를 사용한 PDF 파싱
- 로딩된 문서 페이지 수 확인


In [4]:
# 필요한 라이브러리 임포트
import os
from langchain_community.document_loaders import PyPDFLoader

data_folder = "./data"

# PDF 문서 로딩
def load_specific_pdf_documents(data_folder: str):
    """
    data 폴더 내의 특정 PDF 파일들을 로딩합니다.
    """
    documents = []
    
    # 로딩할 특정 PDF 파일명들
    target_files = [
        "국가별 공공부문 AI 도입 및 활용 전략.pdf",
        "모빌리티_오픈소스_활성화_방안.pdf", 
        "피지컬_AI_현황과_시사점.pdf",
        "AI_인재_양성_정책_현황.pdf"
    ]
    
    # 각 파일을 찾아서 로딩
    for filename in target_files:
        pdf_path = os.path.join(data_folder, filename)
        
        if os.path.exists(pdf_path):
            print(f"📄 로딩 중: {filename}")
            loader = PyPDFLoader(pdf_path)
            file_documents = loader.load()
            documents.extend(file_documents)
        else:
            print(f"⚠️ 파일을 찾을 수 없습니다: {filename}")
    
    print(f"✅ 총 {len(documents)}개의 문서 페이지가 로딩되었습니다.")
    return documents

# PDF 문서 로딩 실행
documents = load_specific_pdf_documents(data_folder)

📄 로딩 중: 국가별 공공부문 AI 도입 및 활용 전략.pdf
📄 로딩 중: 모빌리티_오픈소스_활성화_방안.pdf
📄 로딩 중: 피지컬_AI_현황과_시사점.pdf
📄 로딩 중: AI_인재_양성_정책_현황.pdf
✅ 총 336개의 문서 페이지가 로딩되었습니다.


## 6. 문서 청킹 (Chunking)

로딩된 문서를 검색에 적합한 크기로 분할합니다.
- RecursiveCharacterTextSplitter 사용
- 청크 크기 1000자, 오버랩 200자 설정
- 문장 경계를 고려한 분할


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 청킹 (Chunk 분할)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", ".", " ", ""]
)

# 문서 분할 실행
chunks = text_splitter.split_documents(documents)

print(f"✅ 총 {len(chunks)}개의 청크로 분할되었습니다.")

✅ 총 678개의 청크로 분할되었습니다.


## 7. 하이브리드 벡터 스토어 생성

Dense 벡터와 Sparse 벡터를 함께 사용하는 하이브리드 검색 시스템을 구축합니다.
- Ollama BGE-M3 모델로 dense embedding 생성
- FastEmbed BM25 모델로 sparse embedding 생성
- Qdrant에 하이브리드 컬렉션 생성 및 문서 인덱싱


In [7]:
from uuid import uuid4
from langchain_ollama import OllamaEmbeddings
from langchain_qdrant import FastEmbedSparse, QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance, SparseVectorParams, VectorParams

dense_embeddings = OllamaEmbeddings(model="bge-m3")
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

# # Create a collection with both dense and sparse vectors
# client.create_collection(
#     collection_name="pdf_hybrid",
#     vectors_config={"dense": VectorParams(size=1024, distance=Distance.COSINE)},
#     sparse_vectors_config={
#         "sparse": SparseVectorParams(index=models.SparseIndexParams(on_disk=False))
#     },
# )

qdrant = QdrantVectorStore(
    client=client,
    collection_name="pdf_hybrid",
    embedding=dense_embeddings,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.HYBRID,
    vector_name="dense",
    sparse_vector_name="sparse",
)

# # 문서를 벡터 스토어에 추가
# print("문서를 벡터 스토어에 추가 중...")
# qdrant.add_documents(chunks)
# print(f"✅ {len(chunks)}개의 문서가 Qdrant에 저장되었습니다.")

## 8. Retriever 설정

하이브리드 벡터 스토어에서 검색을 수행할 retriever를 설정합니다.
- 상위 10개의 검색 결과 반환 설정
- 하이브리드 검색 모드로 동작


In [8]:
# 문서로부터 FAISS 인덱스 생성 및 검색기 설정
retriever = qdrant.as_retriever(
    search_kwargs={"k": 10}
)

## 9. 검색 결과 출력 함수

검색 결과를 가독성 있게 출력하기 위한 유틸리티 함수를 정의합니다.
- 문서 출처, 페이지 정보 표시
- 내용 미리보기 제공
- 구분선으로 각 문서 분리


In [9]:
def print_search_results(docs, title):
    """검색 결과를 이쁘게 출력하는 함수"""
    print(f"{title}")
    print("=" * 80)
    print(f"검색된 문서 수: {len(docs)}개\n")
    
    for i, doc in enumerate(docs, 1):
        print(f"📄 문서 {i}")
        print(f"📂 출처: {doc.metadata.get('source', 'N/A')}")
        print(f"📃 페이지: {doc.metadata.get(' page', 'N/A')}")
        print(f"📝 내용 미리보기:")
        print(doc.page_content)
        print("-" * 40)


In [10]:
# 검색 쿼리
query = "우리나라가 AI 인재 수급을 위해 비자를 풀어주나요? 이에 관한 정보를 알려주세요"

# 기본 retriever 검색
basic_docs = retriever.invoke(query)
print_search_results(basic_docs, "🔍 기본 Retriever 검색 결과:")

🔍 기본 Retriever 검색 결과:
검색된 문서 수: 10개

📄 문서 1
📂 출처: ./data\AI_인재_양성_정책_현황.pdf
📃 페이지: N/A
📝 내용 미리보기:
. 더 나아가 현재까지 비교적 관심을 덜 가져온 해외 거주 인재의 활용 방안에 대해서도 심도 있게 고민을 해야 할 것이다.이를 위해 대학·연구기관·기업이 함께 참여하는 산학협력 및 교육 혁신을 더 강화하고, 스타트업에서 대기업까지 폭넓게 활용 가능한 비자 완화와 정주지원, 그리고 국책사업 참여 기회를 대폭 열어두어야 한다. 특히 우리나라 출신의 해외 전문인력에게는 귀환 시 연구나 창업에 필요한 자금·인프라를 집중 지원하고, 귀환하지 않는 인재라도 공동 프로젝트나 자문을 통해 국내에 기여할 수 있는 제도적 장치를 마련해야 한다. 장기적으로는 국제 학술행사 유치와 글로벌 연구센터 설립 등으로 ‘AI 혁신 클러스터’로서의 위상을 높여야 한다.우리나라가 글로벌 AI 인재 유출 심화와 국내 기업의 인재 확보 어려움과 같은 문제를 해소하기 위해서는 본 고에서 제시된 정책들에 대한 검토뿐만 아니라, 제한된 자원을 효율적으로 활용하기 위해 핵심 AI 분야에 '선택과 집중'하고, 인재 유출 방지 및 활용에 대한 ‘패러다임을 전환’하며, 주체 또는 정책간 ‘연계를 통한 시너지 창출’을 더 고민해 나가야 한다. 이러한 종합적인 정책·제도 개선을 통해 우리나라가 과감하고 전략적인 AI 인재 확보·육성에 성공한다면, 미래산업과 국가 경쟁력을 선도하는 발판을 마련할 수 있을 것이다. 
----------------------------------------
📄 문서 2
📂 출처: ./data\AI_인재_양성_정책_현황.pdf
📃 페이지: N/A
📝 내용 미리보기:
SPRi 이슈리포트 IS-203주요국 AI 인재 양성 및 유치 정책 : 현황 및 시사점
9
□(’24.9.26., 법무부), 신(新) 출입국·이민정책추진ㅇ외국 우수 인재 유치를 통해 잠재성장률 및 지역경쟁력 확보, 선별 유입 및 사회통합 촉진으로 이민 